# 4 – Register Processed CSV Dataset with Athena

This notebook registers the processed extreme precipitation dataset
stored in Amazon S3 as an external table in Athena.


## Import Required Libraries and Initialize AWS Session


In [15]:
import boto3
import sagemaker
from sagemaker import get_execution_role
from pyathena import connect
import pandas as pd

sess = sagemaker.Session()
bucket = sess.default_bucket()
region = boto3.Session().region_name
role = get_execution_role()

print("Bucket:", bucket)
print("Region:", region)


Bucket: sagemaker-us-east-1-083422367993
Region: us-east-1


## Define Database and S3 Paths


In [16]:
database_name = "ghcn_extreme_precip_db"
project_prefix = "ghcn-extreme"
processed_prefix = f"{project_prefix}/processed"

csv_s3_location = f"s3://{bucket}/{processed_prefix}/"

print("CSV S3 Location:", csv_s3_location)


CSV S3 Location: s3://sagemaker-us-east-1-083422367993/ghcn-extreme/processed/


## Establish Athena Connection


In [17]:
athena_staging_dir = f"s3://{bucket}/athena/staging/"

conn = connect(
    region_name=region,
    s3_staging_dir=athena_staging_dir
)

print("Connected to Athena.")


Connected to Athena.


## Create External Table for Processed CSV


In [18]:
table_name_csv = "extreme_precip_csv"

csv_s3_location = f"s3://{bucket}/ghcn-extreme/processed_csv/"

create_table_query = f"""
CREATE EXTERNAL TABLE {database_name}.{table_name_csv} (
    station_id STRING,
    date STRING,
    year INT,
    month INT,
    tmax DOUBLE,
    tmin DOUBLE,
    prcp_lag_1 DOUBLE,
    prcp_roll_7 DOUBLE,
    extreme_precip_tomorrow INT
)
ROW FORMAT SERDE 'org.apache.hadoop.hive.serde2.OpenCSVSerde'
WITH SERDEPROPERTIES (
    'separatorChar' = ',',
    'quoteChar' = '\"',
    'escapeChar' = '\\\\'
)
LOCATION '{csv_s3_location}'
TBLPROPERTIES ('skip.header.line.count'='1');
"""

pd.read_sql(create_table_query, conn)

print("CSV external table created.")


/tmp/ipykernel_419/2542186640.py:27: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql(create_table_query, conn)


CSV external table created.


## Verify Table Registration


In [19]:
pd.read_sql(f"SHOW TABLES IN {database_name}", conn)


/tmp/ipykernel_419/818373246.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql(f"SHOW TABLES IN {database_name}", conn)


,tab_name
0,extreme_precip_csv
1,extreme_precip_parquet


## Run Sample Query to Validate Data Access


In [20]:
pd.read_sql(
    f"SELECT * FROM {database_name}.{table_name_csv} LIMIT 10",
    conn
)


/tmp/ipykernel_419/1038048787.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql(


,station_id,date,year,month,tmax,tmin,prcp_lag_1,prcp_roll_7,extreme_precip_tomorrow
0,USW00012921,2006-02-18,2006,2,3.9,-1.1,1.5,0.214286,0
1,USW00012921,2006-02-19,2006,2,5.6,-1.7,0.0,0.257143,0
2,USW00012921,2006-02-20,2006,2,8.9,1.7,0.3,0.400000,0
3,USW00012921,2006-02-21,2006,2,13.9,6.1,1.0,0.442857,0
4,USW00012921,2006-02-22,2006,2,22.2,12.8,0.3,0.442857,0
5,USW00012921,2006-02-23,2006,2,26.1,9.4,0.0,0.442857,0
6,USW00012921,2006-02-24,2006,2,18.9,11.1,0.0,0.228571,0
7,USW00012921,2006-02-25,2006,2,23.3,12.8,0.0,1.714286,0
8,USW00012921,2006-02-26,2006,2,21.7,7.8,10.4,1.671429,0
9,USW00012921,2006-02-27,2006,2,25.0,6.7,0.0,1.528571,0


## Confirm CSV Table is Ready for Parquet Conversion

The CSV dataset is now successfully registered in Athena.
The next step is to convert this dataset to partitioned Parquet format
for improved performance and efficient downstream processing.
